In [ ]:
from mods import readdata as rd
from mods import savedata as sd
from mods import extractdata as ed
from mods import gmap as gm
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

In [2]:
load_dotenv()

True

In [10]:
# 讀取raw data
path = "../../../data/raw/pet_establishment.csv"
df = rd.get_csv_data(path)
df

[✓] CSV檔案已取回



,ID,legaltype,legalname,legaladdress,busitem,animaltype,validnum,validdate,own_name,bos_name,rank_year,rank_code,rank_flag_1,rank_flag_2,rank_text,state_flag
0,15913,A010,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,特寵業字第V1120348號,2027/11/17 上午 12:00:00,宣家瑩,許玲玲,2024.0,B,Y,N,NaN,N
1,19029,A090,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,特寵業繁字第C1140733號,2028/10/2 上午 12:00:00,黃韻蓁,袁子榕,NaN,NaN,NaN,NaN,NaN,N
2,19023,A210,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,特寵業繁字第U1140846號,2028/10/7 上午 12:00:00,施寶美,劉品艾,NaN,NaN,NaN,NaN,NaN,N
3,17813,A060,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,特寵業繁字第S1130016號,2027/3/14 上午 12:00:00,許小芸,許芸鳳、黃信捷,NaN,NaN,NaN,NaN,NaN,N
4,19041,A020,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,特寵業繁字第A1140069號,2028/10/19 上午 12:00:00,洪蔡佳芬,周宜潔,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,17,A040,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,特寵業繁字第W1080095號,2022/1/10 上午 12:00:00,張志雄,彭秀惠,2019.0,B,Y,N,NaN,B
5500,6,A040,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,特寵業繁字第W1080128號,2022/3/14 上午 12:00:00,詹東旺,曾健倫,2019.0,B,Y,N,NaN,P
5501,4,A040,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,特寵業繁字第W1070135號,2021/11/16 上午 12:00:00,周張月琴,周進欽,2019.0,B,Y,N,NaN,P
5502,3,A040,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,特寵業字第Ｗ1070144號,2024/9/25 上午 12:00:00,吳慧珠,龔高益,2019.0,C,Y,N,NaN,P


In [11]:
# 執行ETL
# 去除不必要的欄位
df_filtered = df[["legalname", "legaladdress", "busitem", "animaltype", "state_flag"]]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
1,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,N
2,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N
3,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N
4,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N
...,...,...,...,...,...
5499,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,B
5500,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,P
5501,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,P
5502,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,P


In [12]:
# busitem = C表示寄養，state_flag = N表示營業中
df_filtered = df_filtered[
    (df_filtered["busitem"].str.contains("C")) & (df_filtered["state_flag"] == "N")
]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
2,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N
3,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N
4,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N
5,東森寵物雲股份有限公司花蓮玉里分公司,花蓮縣玉里鎮和平路96號,C,狗、貓,N
...,...,...,...,...,...
5368,假日貓坊,桃園市龍潭區百年里百年路33號,ABC,貓,N
5420,貓屋住宿用品店,雲林縣斗六市永安路77號,C,貓,N
5424,威地犬舍,雲林縣虎尾鎮安溪路121-6號,ABC,狗,N
5452,東森寵物雲股份有限公司南屯文心分公司,臺中市南屯區文心路一段459號,BC,狗、貓,N


In [13]:
# 找出屬於六都的機構
df_filtered["city"], df_filtered["district"] = zip(
    *df_filtered["legaladdress"].apply(ed.extract_city_district)
)
df_filtered = df_filtered[df_filtered["city"].notna()].reset_index(drop=True)
df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_408\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(
C:\Users\dinni\AppData\Local\Temp\ipykernel_408\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(


,legalname,legaladdress,busitem,animaltype,state_flag,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N,臺北市,大安區
1,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N,臺南市,永康區
2,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N,臺中市,南區
3,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N,新北市,板橋區
4,牛寶法鬥犬舍,高雄市林園區龍潭路225巷18號,ABC,狗,N,高雄市,林園區
...,...,...,...,...,...,...,...
2090,貓的旅坊,臺南市東區裕農路691號1-2樓,C,貓,N,臺南市,東區
2091,美樂犬莊,臺南市東山區東中里16鄰新東路二段50號,ABC,狗,N,臺南市,東山區
2092,假日貓坊,桃園市龍潭區百年里百年路33號,ABC,貓,N,桃園市,龍潭區
2093,東森寵物雲股份有限公司南屯文心分公司,臺中市南屯區文心路一段459號,BC,狗、貓,N,臺中市,南屯區


In [14]:
# 搜尋名字有旅、住、宿的資料
df_filtered = df_filtered[df_filtered["legalname"].str.contains("旅|住|宿", regex=True)]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag,city,district
47,緹米貓旅館,新北市永和區福和路167號2樓,C,貓,N,新北市,永和區
53,瑞發財貓旅館,臺南市南區金華路一段468號1-2樓,BC,貓,N,臺南市,南區
58,吾悠喵旅生活館,新北市中和區中正路210號1樓,C,貓,N,新北市,中和區
66,樂牧寵物旅館,新北市新莊區建興街20號1樓,BC,狗、貓,N,新北市,新莊區
68,糖果屋貓咪旅館,新北市板橋區大觀路2段7巷24號,C,貓,N,新北市,板橋區
...,...,...,...,...,...,...,...
1997,狗狗打呼寵物旅館,臺中市西屯區臺中市西屯區青海路一段194號2樓,C,狗,N,臺中市,西屯區
2011,阿尼牧絲寵物旅館,臺中市烏日區臺中市烏日區高鐵路一段12號1樓、2樓、3樓,C,狗、貓,N,臺中市,烏日區
2048,貝果小貓旅店,臺中市烏日區環河北路258號,C,貓,N,臺中市,烏日區
2053,柴町寵物旅舍,桃園市桃園區中正里22鄰文中路10號,C,狗,N,桃園市,桃園區


In [15]:
# 只留下名稱、地址、市和區
df_filtered = df_filtered[["legalname", "legaladdress", "city", "district"]]
df_filtered

,legalname,legaladdress,city,district
47,緹米貓旅館,新北市永和區福和路167號2樓,新北市,永和區
53,瑞發財貓旅館,臺南市南區金華路一段468號1-2樓,臺南市,南區
58,吾悠喵旅生活館,新北市中和區中正路210號1樓,新北市,中和區
66,樂牧寵物旅館,新北市新莊區建興街20號1樓,新北市,新莊區
68,糖果屋貓咪旅館,新北市板橋區大觀路2段7巷24號,新北市,板橋區
...,...,...,...,...
1997,狗狗打呼寵物旅館,臺中市西屯區臺中市西屯區青海路一段194號2樓,臺中市,西屯區
2011,阿尼牧絲寵物旅館,臺中市烏日區臺中市烏日區高鐵路一段12號1樓、2樓、3樓,臺中市,烏日區
2048,貝果小貓旅店,臺中市烏日區環河北路258號,臺中市,烏日區
2053,柴町寵物旅舍,桃園市桃園區中正里22鄰文中路10號,桃園市,桃園區


In [16]:
# 修改欄位名稱
df_filtered = df_filtered.rename(
    columns={
        "legalname": "name",
        "legaladdress": "address",
    }
)
df_filtered

,name,address,city,district
47,緹米貓旅館,新北市永和區福和路167號2樓,新北市,永和區
53,瑞發財貓旅館,臺南市南區金華路一段468號1-2樓,臺南市,南區
58,吾悠喵旅生活館,新北市中和區中正路210號1樓,新北市,中和區
66,樂牧寵物旅館,新北市新莊區建興街20號1樓,新北市,新莊區
68,糖果屋貓咪旅館,新北市板橋區大觀路2段7巷24號,新北市,板橋區
...,...,...,...,...
1997,狗狗打呼寵物旅館,臺中市西屯區臺中市西屯區青海路一段194號2樓,臺中市,西屯區
2011,阿尼牧絲寵物旅館,臺中市烏日區臺中市烏日區高鐵路一段12號1樓、2樓、3樓,臺中市,烏日區
2048,貝果小貓旅店,臺中市烏日區環河北路258號,臺中市,烏日區
2053,柴町寵物旅舍,桃園市桃園區中正里22鄰文中路10號,桃園市,桃園區


In [ ]:
# 透過google api並傳送名稱與地址取得place_id
API_KEY = os.getenv("GOOGLE_MAP_KEY_CHGWYELLOW")
result = []

# enumerate會自動將被iterate的物件附上index
for i, (idx, row) in enumerate(df_filtered.iterrows()):
    query = f"{row['name']} {row['address']}"

    result.append(gm.get_place_id(API_KEY, query))
df_filtered["place_id"] = np.nan
df_filtered.loc[:, "place_id"] = result
df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_408\1597705559.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['ChIJcXJcgHSpQjQRNDlpI-2FZdI', 'ChIJY2gsl0J1bjQRImL6pDP8rY0', 'ChIJpfglnWcDaDQROqnauibQ4pE', 'ChIJWde5Zsvni2MRU7nv4bWTcgM', 'ChIJHcIvT6ypQjQRc9gESCt3ZSY', 'ChIJ6bxHpL6pQjQRxT3sIsWGHHQ', 'ChIJlZ49HAd3bjQRVI-hqM6Csec', 'ChIJp6ulzdofaDQRabB4RPtieSg', 'ChIJgXY55DQ9aTQRcHNGqavnT00', 'ChIJr8ISuASqQjQROYYmrkk_kdo', 'ChIJLVJZNjEdaDQRFv3zliGQvys', 'ChIJe1bS3SKoQjQRWbq1T-LgvD4', 'ChIJT6_obDYPbjQRD302Ipqcqyg', 'ChIJe4YFaACpQjQR2HKiVrJ87sE', 'ChIJhZk351Q8aTQR1K8s_YrCVMM', 'ChIJvapPIRYFbjQRYGFMsxcehgM', 'ChIJP5JrgUoiaDQRrpVQ4PapsD8', 'ChIJ4RQ6mX8FbjQRUsnNU65hy-Y', 'ChIJ561iVFQ8aTQRw8diMoHwU5w', 'ChIJvwYbBXQRaTQRY0y7pZgr-QY', 'ChIJqazxFIMfaDQRBrkmvDtYf7M', 'ChIJQ90SF30baDQRDhhCkm5QzU0', 'ChIJR4JtH42pQjQR0IsAAXV-kas', 'ChIJI4ikpxcXaTQRoPt_oh_Pgck', 'ChIJqVkEw-p7bjQRsoxoM4Sz28c', 'ChIJcwLVUV2rQjQRe_yzF_mMEdA', 'ChIJuYsH

In [ ]:
# 透過place_id找到詳細資料
result = []
for _, row in df_filtered.iterrows():
    result.append(gm.gmap_info(row["name"], API_KEY, row["place_id"]))
df_checked = pd.DataFrame(result)
df_checked

,name,place_id,business_status,address,phone,opening_hours,rating,rating_total,longitude,latitude,map_url,newest_review
0,WE CAT喵旅/貓旅館 、美容、安親、用品,ChIJcXJcgHSpQjQRNDlpI-2FZdI,OPERATIONAL,23450台灣新北市永和區福和路167號2 F,0909578920,"[星期一: 11:30 – 19:30, 星期二: 11:30 – 19:30, 星期三: ...",5.0,52.0,121.520319,25.007589,https://maps.google.com/?cid=15160670974160484660,2023-01-01
1,凱瑞貓鐵-貓咪/貓咪住宿/貓咪安親/緬因貓/寵物/寵物食品/寵物用品,ChIJY2gsl0J1bjQRImL6pDP8rY0,OPERATIONAL,70251台灣臺南市南區金華路一段468號,None,"[星期一: 10:00 – 01:00, 星期二: 10:00 – 20:00, 星期三: ...",NaN,NaN,120.190917,22.972598,https://maps.google.com/?cid=10209093229060186658,None
2,吾悠喵旅-純貓咪旅館（預約制）,ChIJpfglnWcDaDQROqnauibQ4pE,OPERATIONAL,235台灣新北市中和區中正路210號,None,"[星期一: 12:00 – 19:30, 星期二: 12:00 – 19:30, 星期三: ...",5.0,28.0,121.496768,24.994021,https://maps.google.com/?cid=10512193344998385978,2024-02-23
3,樂牧寵物旅館,ChIJWde5Zsvni2MRU7nv4bWTcgM,OPERATIONAL,242台灣新北市新莊區建興街20號,0981668107,"[星期一: 11:00 – 21:00, 星期二: 11:00 – 21:00, 星期三: ...",5.0,152.0,121.450124,25.036929,https://maps.google.com/?cid=248423338847746387,2024-09-25
4,糖果屋貓咪旅館 - 板橋寵物貓住宿| 甜蜜250 時尚350 | 長期寄養包月 | 長住貓旅...,ChIJHcIvT6ypQjQRc9gESCt3ZSY,OPERATIONAL,220台灣新北市板橋區大觀路二段7巷24號,0985988562,"[星期一: 09:00 – 21:00, 星期二: 09:00 – 21:00, 星期三: ...",4.6,34.0,121.447887,25.003331,https://maps.google.com/?cid=2766748573864614003,2023-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...
165,狗狗打呼寵物旅館,ChIJi4BFR-g9aTQRhQnGAY47D40,OPERATIONAL,407053台灣臺中市西屯區青海路一段194號2樓,0436035298,"[星期一: 休息, 星期二: 11:00 – 19:30, 星期三: 11:00 – 19:...",5.0,45.0,120.652484,24.165819,https://maps.google.com/?cid=10164408365099649413,2025-07-19
166,阿尼牧絲寵物旅館,ChIJr8v8PjQ_aTQRvlf2GZNHJRk,OPERATIONAL,414台灣臺中市烏日區高鐵路一段12號,0921016857,"[星期一: 11:00 – 20:00, 星期二: 11:00 – 20:00, 星期三: ...",5.0,10.0,120.601833,24.106039,https://maps.google.com/?cid=1811933122207831998,2023-07-06
167,貝果犬舍,ChIJF_IK6hk7aTQRSqxEcXUjOOA,OPERATIONAL,41352台灣臺中市霧峰區林森路856-1號,0985800052,"[星期一: 08:00 – 17:00, 星期二: 08:00 – 17:00, 星期三: ...",5.0,1.0,120.696569,24.068607,https://maps.google.com/?cid=16156702650510126154,2025-09-25
168,柴町寵物旅舍(全預約制),ChIJuSDQvxsfaDQRe86i7FuNdQU,OPERATIONAL,330台灣桃園市桃園區文中路10號,033601313,"[星期一: 09:30 – 20:00, 星期二: 09:30 – 20:00, 星期三: ...",4.9,106.0,121.294454,24.996163,https://maps.google.com/?cid=393375968416419451,2025-06-16


In [15]:
df_merged = df_filtered.merge(
    df_checked,
    how="outer",
    left_on=df_filtered["place_id"],
    right_on=df_checked["place_id"],
    suffixes=["_filtered", "_checked"]
)
df_merged = df_merged.drop(columns=["place_id_filtered", "place_id_checked"])
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,rating,rating_total,longitude,latitude,map_url,newest_review
0,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,2025-06-09
1,ChIJ-ZHI8pMFbjQRt7zP9EVOy9g,我的狗寵物美容住宿,高雄市三民區高雄市三民區民族一路525號,高雄市,三民區,My Dog 寵物美容．住宿,OPERATIONAL,807台灣高雄市三民區民族一路525號,073501623,"[星期一: 11:00 – 19:00, 星期二: 休息, 星期三: 11:00 – 19:...",4.9,78.0,120.316272,22.665565,https://maps.google.com/?cid=15621665794831662263,2022-01-22
2,ChIJ-y0efGc9aTQRs36czlSKmeM,醬棒貓旅工作室,臺中市西區自由路一段28-5號1樓,臺中市,西區,醬棒貓旅工作室,OPERATIONAL,408034台灣臺中市南屯區大業路355號4樓,0919828148,None,5.0,45.0,120.646004,24.153503,https://maps.google.com/?cid=16400291614894620339,2025-06-02
3,ChIJ0VXEl91TXTQRdE0BddjV5xI,橘舍寵物沙龍旅館,新北市汐止區秀峰路144號1樓,新北市,汐止區,洗隻狗旅館 XIZHI DOG,OPERATIONAL,221台灣新北市汐止區秀峰路144號,0226489099,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 21:00, 星期三: ...",4.8,33.0,121.660033,25.062550,https://maps.google.com/?cid=1362292537955536244,2025-04-26
4,ChIJ1754PUsgaDQRhy5J6Kpi9-k,達特歐寵物旅館,桃園市大園區五權里五青路183巷53號,桃園市,大園區,DOTEL寵物旅館,OPERATIONAL,337台灣桃園市大園區五青路183巷53號,None,"[星期一: 10:00 – 18:00, 星期二: 10:00 – 18:00, 星期三: ...",4.5,41.0,121.235822,25.018950,https://maps.google.com/?cid=16859052216265813639,2019-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,ChIJw02KsiZ3bjQR6kgZLQ5btUM,芥菜籽寵物輕旅店,臺南市永康區東橋里9鄰東橋十街32號3樓,臺南市,永康區,芥菜種動物醫院(中西整合醫學長照安養中心),OPERATIONAL,710台灣臺南市永康區東橋十街32號,063027620,"[星期一: 09:00 – 16:45, 18:00 – 20:30, 星期二: 09:00...",4.7,206.0,120.232118,23.021750,https://maps.google.com/?cid=4878905887770101994,2024-02-24
182,ChIJwd25h2OnQjQRnUs3l3e9iG4,狗趣兒寵物旅館,新北市林口區林口區文化三路1段617巷163號1樓,新北市,林口區,GoChill狗趣兒寵物旅館,OPERATIONAL,"163號, 文化三路一段617巷林口區新北市台灣 244015",0226086782,"[星期一: 09:00 – 20:00, 星期二: 09:00 – 20:00, 星期三: ...",4.5,17.0,121.371398,25.077973,https://maps.google.com/?cid=7964824262340463517,2023-04-08
183,ChIJxVmJ884FbjQRzk3Q7r5wO0g,蓁心好寵旅店,高雄市鳳山區新富路10號,高雄市,鳳山區,蓁心寵物貓狗美容住宿/鳯山寵物住宿/鳯山寵物美容SPA/苓雅區寵物美容/苓雅區寵物住宿/前鎮...,OPERATIONAL,830台灣高雄市鳳山區新富路10號,0981779770,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...",4.6,73.0,120.333171,22.612219,https://maps.google.com/?cid=5204877759709466062,2020-06-04
184,ChIJy8aWEpF1bjQR1u2uoUdA20U,月光貓旅館,臺南市東區崇善路565號3樓,臺南市,東區,月光貓旅 Moonlight Cat Inn,OPERATIONAL,70169台灣臺南市東區崇善路565號3樓,062901369,"[星期一: 11:00 – 20:00, 星期二: 11:00 – 20:00, 星期三: ...",4.9,24.0,120.229807,22.972721,https://maps.google.com/?cid=5033687684916112854,2021-01-10


In [ ]:
# 檢查是否有空值
df_merged.isna().any()

name               False
place_id           False
business_status     True
address            False
phone               True
opening_hours       True
rating              True
rating_total        True
longitude          False
latitude           False
map_url            False
newest_review       True
dtype: bool

In [3]:
df = rd.get_csv_data("../../../data/processed/pet_hotel_ETL.csv")
df

[✓] CSV檔案已取回



,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,rating,rating_total,longitude,latitude,map_url,newest_review
0,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,928409122.0,"['星期一: 13:00 – 19:00', '星期二: 13:00 – 19:00', '...",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,2025-06-09
1,ChIJ-ZHI8pMFbjQRt7zP9EVOy9g,我的狗寵物美容住宿,高雄市三民區高雄市三民區民族一路525號,高雄市,三民區,My Dog 寵物美容．住宿,OPERATIONAL,807台灣高雄市三民區民族一路525號,73501623.0,"['星期一: 11:00 – 19:00', '星期二: 休息', '星期三: 11:00 ...",4.9,78.0,120.316272,22.665565,https://maps.google.com/?cid=15621665794831662263,2022-01-22
2,ChIJ-y0efGc9aTQRs36czlSKmeM,醬棒貓旅工作室,臺中市西區自由路一段28-5號1樓,臺中市,西區,醬棒貓旅工作室,OPERATIONAL,408034台灣臺中市南屯區大業路355號4樓,919828148.0,NaN,5.0,45.0,120.646004,24.153503,https://maps.google.com/?cid=16400291614894620339,2025-06-02
3,ChIJ0VXEl91TXTQRdE0BddjV5xI,橘舍寵物沙龍旅館,新北市汐止區秀峰路144號1樓,新北市,汐止區,洗隻狗旅館 XIZHI DOG,OPERATIONAL,221台灣新北市汐止區秀峰路144號,226489099.0,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 21:00', '...",4.8,33.0,121.660033,25.062550,https://maps.google.com/?cid=1362292537955536244,2025-04-26
4,ChIJ1754PUsgaDQRhy5J6Kpi9-k,達特歐寵物旅館,桃園市大園區五權里五青路183巷53號,桃園市,大園區,DOTEL寵物旅館,OPERATIONAL,337台灣桃園市大園區五青路183巷53號,NaN,"['星期一: 10:00 – 18:00', '星期二: 10:00 – 18:00', '...",4.5,41.0,121.235822,25.018950,https://maps.google.com/?cid=16859052216265813639,2019-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,ChIJw02KsiZ3bjQR6kgZLQ5btUM,芥菜籽寵物輕旅店,臺南市永康區東橋里9鄰東橋十街32號3樓,臺南市,永康區,芥菜種動物醫院(中西整合醫學長照安養中心),OPERATIONAL,710台灣臺南市永康區東橋十街32號,63027620.0,"['星期一: 09:00 – 16:45, 18:00 – 20:30', '星期二: 09...",4.7,206.0,120.232118,23.021750,https://maps.google.com/?cid=4878905887770101994,2024-02-24
182,ChIJwd25h2OnQjQRnUs3l3e9iG4,狗趣兒寵物旅館,新北市林口區林口區文化三路1段617巷163號1樓,新北市,林口區,GoChill狗趣兒寵物旅館,OPERATIONAL,"163號, 文化三路一段617巷林口區新北市台灣 244015",226086782.0,"['星期一: 09:00 – 20:00', '星期二: 09:00 – 20:00', '...",4.5,17.0,121.371398,25.077973,https://maps.google.com/?cid=7964824262340463517,2023-04-08
183,ChIJxVmJ884FbjQRzk3Q7r5wO0g,蓁心好寵旅店,高雄市鳳山區新富路10號,高雄市,鳳山區,蓁心寵物貓狗美容住宿/鳯山寵物住宿/鳯山寵物美容SPA/苓雅區寵物美容/苓雅區寵物住宿/前鎮...,OPERATIONAL,830台灣高雄市鳳山區新富路10號,981779770.0,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 19:00', '...",4.6,73.0,120.333171,22.612219,https://maps.google.com/?cid=5204877759709466062,2020-06-04
184,ChIJy8aWEpF1bjQR1u2uoUdA20U,月光貓旅館,臺南市東區崇善路565號3樓,臺南市,東區,月光貓旅 Moonlight Cat Inn,OPERATIONAL,70169台灣臺南市東區崇善路565號3樓,62901369.0,"['星期一: 11:00 – 20:00', '星期二: 11:00 – 20:00', '...",4.9,24.0,120.229807,22.972721,https://maps.google.com/?cid=5033687684916112854,2021-01-10


In [4]:
revised_columns = [
    "key_0",
    "name_filtered",
    "name_checked",
    "address_filtered",
    "address_checked",
    "phone",
    "city",
    "district",
    "business_status",
    "opening_hours",
    "rating",
    "rating_total",
    "longitude",
    "latitude",
    "map_url",
    "newest_review",
]
df = df[revised_columns]
df

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,rating,rating_total,longitude,latitude,map_url,newest_review
0,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,928409122.0,高雄市,三民區,OPERATIONAL,"['星期一: 13:00 – 19:00', '星期二: 13:00 – 19:00', '...",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,2025-06-09
1,ChIJ-ZHI8pMFbjQRt7zP9EVOy9g,我的狗寵物美容住宿,My Dog 寵物美容．住宿,高雄市三民區高雄市三民區民族一路525號,807台灣高雄市三民區民族一路525號,73501623.0,高雄市,三民區,OPERATIONAL,"['星期一: 11:00 – 19:00', '星期二: 休息', '星期三: 11:00 ...",4.9,78.0,120.316272,22.665565,https://maps.google.com/?cid=15621665794831662263,2022-01-22
2,ChIJ-y0efGc9aTQRs36czlSKmeM,醬棒貓旅工作室,醬棒貓旅工作室,臺中市西區自由路一段28-5號1樓,408034台灣臺中市南屯區大業路355號4樓,919828148.0,臺中市,西區,OPERATIONAL,NaN,5.0,45.0,120.646004,24.153503,https://maps.google.com/?cid=16400291614894620339,2025-06-02
3,ChIJ0VXEl91TXTQRdE0BddjV5xI,橘舍寵物沙龍旅館,洗隻狗旅館 XIZHI DOG,新北市汐止區秀峰路144號1樓,221台灣新北市汐止區秀峰路144號,226489099.0,新北市,汐止區,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 21:00', '...",4.8,33.0,121.660033,25.062550,https://maps.google.com/?cid=1362292537955536244,2025-04-26
4,ChIJ1754PUsgaDQRhy5J6Kpi9-k,達特歐寵物旅館,DOTEL寵物旅館,桃園市大園區五權里五青路183巷53號,337台灣桃園市大園區五青路183巷53號,NaN,桃園市,大園區,OPERATIONAL,"['星期一: 10:00 – 18:00', '星期二: 10:00 – 18:00', '...",4.5,41.0,121.235822,25.018950,https://maps.google.com/?cid=16859052216265813639,2019-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,ChIJw02KsiZ3bjQR6kgZLQ5btUM,芥菜籽寵物輕旅店,芥菜種動物醫院(中西整合醫學長照安養中心),臺南市永康區東橋里9鄰東橋十街32號3樓,710台灣臺南市永康區東橋十街32號,63027620.0,臺南市,永康區,OPERATIONAL,"['星期一: 09:00 – 16:45, 18:00 – 20:30', '星期二: 09...",4.7,206.0,120.232118,23.021750,https://maps.google.com/?cid=4878905887770101994,2024-02-24
182,ChIJwd25h2OnQjQRnUs3l3e9iG4,狗趣兒寵物旅館,GoChill狗趣兒寵物旅館,新北市林口區林口區文化三路1段617巷163號1樓,"163號, 文化三路一段617巷林口區新北市台灣 244015",226086782.0,新北市,林口區,OPERATIONAL,"['星期一: 09:00 – 20:00', '星期二: 09:00 – 20:00', '...",4.5,17.0,121.371398,25.077973,https://maps.google.com/?cid=7964824262340463517,2023-04-08
183,ChIJxVmJ884FbjQRzk3Q7r5wO0g,蓁心好寵旅店,蓁心寵物貓狗美容住宿/鳯山寵物住宿/鳯山寵物美容SPA/苓雅區寵物美容/苓雅區寵物住宿/前鎮...,高雄市鳳山區新富路10號,830台灣高雄市鳳山區新富路10號,981779770.0,高雄市,鳳山區,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 19:00', '...",4.6,73.0,120.333171,22.612219,https://maps.google.com/?cid=5204877759709466062,2020-06-04
184,ChIJy8aWEpF1bjQR1u2uoUdA20U,月光貓旅館,月光貓旅 Moonlight Cat Inn,臺南市東區崇善路565號3樓,70169台灣臺南市東區崇善路565號3樓,62901369.0,臺南市,東區,OPERATIONAL,"['星期一: 11:00 – 20:00', '星期二: 11:00 – 20:00', '...",4.9,24.0,120.229807,22.972721,https://maps.google.com/?cid=5033687684916112854,2021-01-10


In [5]:
if df["key_0"].duplicated().any():
    df = df.drop_duplicates(subset="key_0")
df

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,rating,rating_total,longitude,latitude,map_url,newest_review
0,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,928409122.0,高雄市,三民區,OPERATIONAL,"['星期一: 13:00 – 19:00', '星期二: 13:00 – 19:00', '...",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,2025-06-09
1,ChIJ-ZHI8pMFbjQRt7zP9EVOy9g,我的狗寵物美容住宿,My Dog 寵物美容．住宿,高雄市三民區高雄市三民區民族一路525號,807台灣高雄市三民區民族一路525號,73501623.0,高雄市,三民區,OPERATIONAL,"['星期一: 11:00 – 19:00', '星期二: 休息', '星期三: 11:00 ...",4.9,78.0,120.316272,22.665565,https://maps.google.com/?cid=15621665794831662263,2022-01-22
2,ChIJ-y0efGc9aTQRs36czlSKmeM,醬棒貓旅工作室,醬棒貓旅工作室,臺中市西區自由路一段28-5號1樓,408034台灣臺中市南屯區大業路355號4樓,919828148.0,臺中市,西區,OPERATIONAL,NaN,5.0,45.0,120.646004,24.153503,https://maps.google.com/?cid=16400291614894620339,2025-06-02
3,ChIJ0VXEl91TXTQRdE0BddjV5xI,橘舍寵物沙龍旅館,洗隻狗旅館 XIZHI DOG,新北市汐止區秀峰路144號1樓,221台灣新北市汐止區秀峰路144號,226489099.0,新北市,汐止區,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 21:00', '...",4.8,33.0,121.660033,25.062550,https://maps.google.com/?cid=1362292537955536244,2025-04-26
4,ChIJ1754PUsgaDQRhy5J6Kpi9-k,達特歐寵物旅館,DOTEL寵物旅館,桃園市大園區五權里五青路183巷53號,337台灣桃園市大園區五青路183巷53號,NaN,桃園市,大園區,OPERATIONAL,"['星期一: 10:00 – 18:00', '星期二: 10:00 – 18:00', '...",4.5,41.0,121.235822,25.018950,https://maps.google.com/?cid=16859052216265813639,2019-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,ChIJw02KsiZ3bjQR6kgZLQ5btUM,芥菜籽寵物輕旅店,芥菜種動物醫院(中西整合醫學長照安養中心),臺南市永康區東橋里9鄰東橋十街32號3樓,710台灣臺南市永康區東橋十街32號,63027620.0,臺南市,永康區,OPERATIONAL,"['星期一: 09:00 – 16:45, 18:00 – 20:30', '星期二: 09...",4.7,206.0,120.232118,23.021750,https://maps.google.com/?cid=4878905887770101994,2024-02-24
182,ChIJwd25h2OnQjQRnUs3l3e9iG4,狗趣兒寵物旅館,GoChill狗趣兒寵物旅館,新北市林口區林口區文化三路1段617巷163號1樓,"163號, 文化三路一段617巷林口區新北市台灣 244015",226086782.0,新北市,林口區,OPERATIONAL,"['星期一: 09:00 – 20:00', '星期二: 09:00 – 20:00', '...",4.5,17.0,121.371398,25.077973,https://maps.google.com/?cid=7964824262340463517,2023-04-08
183,ChIJxVmJ884FbjQRzk3Q7r5wO0g,蓁心好寵旅店,蓁心寵物貓狗美容住宿/鳯山寵物住宿/鳯山寵物美容SPA/苓雅區寵物美容/苓雅區寵物住宿/前鎮...,高雄市鳳山區新富路10號,830台灣高雄市鳳山區新富路10號,981779770.0,高雄市,鳳山區,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 19:00', '...",4.6,73.0,120.333171,22.612219,https://maps.google.com/?cid=5204877759709466062,2020-06-04
184,ChIJy8aWEpF1bjQR1u2uoUdA20U,月光貓旅館,月光貓旅 Moonlight Cat Inn,臺南市東區崇善路565號3樓,70169台灣臺南市東區崇善路565號3樓,62901369.0,臺南市,東區,OPERATIONAL,"['星期一: 11:00 – 20:00', '星期二: 11:00 – 20:00', '...",4.9,24.0,120.229807,22.972721,https://maps.google.com/?cid=5033687684916112854,2021-01-10


In [6]:
# 儲存ETL後CSV檔
path = "../../../data/processed/pet_hotel_ETL.csv"
sd.store_to_csv_no_index(df, path)

[✓] 不含index的CSV檔已存檔完畢
